## YOLO training + Offside Logic

In [1]:
import cv2
import numpy as np
from SoccerNet.Downloader import SoccerNetDownloader
import json

#### Dataset

In [ ]:
# Download SoccerNet-v3 Mini Dataset
local_directory = "G:\\CV_Dataset_full"
downloader = SoccerNetDownloader(LocalDirectory=local_directory)
downloader.downloadGames(files=["Labels-v3.json", "Frames-v3.zip"], split=["train","valid","test"], task="frames")

#### Folder Organization

In [ ]:
import os
import zipfile
import json
import random
import numpy as np
from pathlib import Path
from shutil import copyfile

# CONFIG
ROOT_DIR = Path(r"G:\\CV_dataset_short")
OUT_DIR  = Path(r"G:\\yolo_offside_short")

CLASS_MAP = {
    "Player team left":           0,
    "Player team right":          1,
    "Goalkeeper team left":       2,
    "Goalkeeper team right":      3,
    "Side line top":              4,
    "Side line bottom":           5,
    "Side line left":             6,
    "Side line right":            7,
    "Big rect. left main":        8,
    "Big rect. left top":         9,
    "Big rect. left bottom":     10,
    "Big rect. right main":      11,
    "Big rect. right top":       12,
    "Big rect. right bottom":    13,
    "Small rect. left main":     14,
    "Small rect. right main":    15,
}

SPLITS = {"train":0.8, "val":0.1, "test":0.1}
assert abs(sum(SPLITS.values()) - 1.0) < 1e-6

# HELPERS 
def yolo_box(x1,y1,x2,y2,W,H):
    xc = ((x1+x2)/2)/W
    yc = ((y1+y2)/2)/H
    w_ = (x2-x1)/W
    h_ = (y2-y1)/H
    return xc,yc,w_,h_

def ensure_dirs(split):
    (OUT_DIR/split/"images").mkdir(parents=True, exist_ok=True)
    (OUT_DIR/split/"labels").mkdir(parents=True, exist_ok=True)

#  MAIN
if __name__ == "__main__":
    # 1. Find all matches
    matches = []
    for league in ROOT_DIR.iterdir():
        if not league.is_dir(): continue
        for season in league.iterdir():
            if not season.is_dir(): continue
            for match in season.iterdir():
                z = match/"Frames-v3.zip"
                l = match/"Labels-v3.json"
                if z.exists() and l.exists():
                    matches.append((z,l))
    print(f"Found {len(matches)} matches")

    # 2. Unzip & collect frames
    all_items = []
    for zipf, labelf in matches:
        frames_dir = zipf.parent/f"frames_{zipf.stem}"
        if not frames_dir.exists():
            with zipfile.ZipFile(zipf, "r") as zf:
                zf.extractall(frames_dir)

        data = json.load(open(labelf, "r"))["actions"]
        for fname, anno in data.items():
            imgp = frames_dir/fname
            if imgp.exists():
                all_items.append((imgp, anno))
    print(f"Collected {len(all_items)} total frames")

    # 3. Split
    random.shuffle(all_items)
    n = len(all_items)
    i1 = int(n * SPLITS["train"])
    i2 = i1 + int(n * SPLITS["val"])
    splits = {
        "train": all_items[:i1],
        "val":   all_items[i1:i2],
        "test":  all_items[i2:]
    }
    print({k: len(v) for k,v in splits.items()})

    # 4. Prep directories
    for sp in splits:
        ensure_dirs(sp)

    # 5. Write images+labels
    for sp, items in splits.items():
        for imgp, anno in items:
            meta = anno["imageMetadata"]
            W, H = meta["width"], meta["height"]
            subdir = Path(meta["localpath"])

            # a) Copy image into <localpath>
            dst_img = OUT_DIR/sp/"images"/subdir/imgp.name
            dst_img.parent.mkdir(parents=True, exist_ok=True)
            copyfile(imgp, dst_img)

            # b) write its label .txt alongside
            dst_lbl = OUT_DIR/sp/"labels"/subdir/imgp.with_suffix(".txt").name
            dst_lbl = (OUT_DIR/sp/"labels"/subdir)/(imgp.stem + ".txt")
            dst_lbl.parent.mkdir(parents=True, exist_ok=True)
            with open(dst_lbl, "w") as f:
                # Players
                for bb in anno["bboxes"]:
                    cls = bb["class"]
                    if cls not in CLASS_MAP: continue
                    cid = CLASS_MAP[cls]
                    x1,y1 = bb["points"]["x1"], bb["points"]["y1"]
                    x2,y2 = bb["points"]["x2"], bb["points"]["y2"]
                    xc,yc,w_,h_ = yolo_box(x1,y1,x2,y2,W,H)
                    f.write(f"{cid} {xc:.6f} {yc:.6f} {w_:.6f} {h_:.6f}\n")
                # Lines & rects
                for ln in anno["lines"]:
                    cls = ln["class"]
                    if cls not in CLASS_MAP: continue
                    cid = CLASS_MAP[cls]
                    pts = np.array(ln["points"],float).reshape(-1,2)
                    xs, ys = pts[:,0], pts[:,1]
                    x1,y1 = xs.min(), ys.min()
                    x2,y2 = xs.max(), ys.max()
                    xc,yc,w_,h_ = yolo_box(x1,y1,x2,y2,W,H)
                    f.write(f"{cid} {xc:.6f} {yc:.6f} {w_:.6f} {h_:.6f}\n")

    print("✅ Done - dataset with preserved folders is in", OUT_DIR)


Found 87 matches
Collected 2433 total frames
{'train': 1946, 'val': 243, 'test': 244}
✅ Done – dataset with preserved folders is in G:\yolo_offside_short


#### YOLO Training

In [ ]:
from ultralytics import YOLO

# 1. load a pre-trained YOLOv8 model
model = YOLO('yolov8n.pt')   # nano model, super-fast

# 2. Train
model.train(
    data='G:\yolo_offside_short\dataset.yaml',     # the data config file you just made
    epochs=50,            # adjust to taste
    imgsz=640,            # resizing to 640×640
    batch=16,             # batch size (reduce if you run out of GPU RAM)
    device=0,             # GPU 0 (set to 'cpu' if you don’t have a GPU)
    project='runs/train', # where to save logs / weights
    name='offsidev3',       # subfolder for this experiment
    save=True,            # save weights at end of training
    verbose=True
)


#### YOLO Detected Boxes

In [17]:
# Show YOLO detection boxes
from ultralytics import YOLO
import cv2

# 1. Load your trained offside model
model = YOLO('runs/train/offsidev32/weights/best.pt')

# 2. Path to a sample image
img_path = '6.png'  # change to your file

# 3. Read it with OpenCV
img = cv2.imread(img_path)
assert img is not None, f"Could not read {img_path}"

# 4. Inference (returns a list of Results, here just 1)
results = model(img, conf=0.25, imgsz=640)

# 5. Draw every box
res = results[0]
for box in res.boxes:
    # box.xyxy: tensor of shape (n,4), box.cls, box.conf
    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
    cls_id         = int(box.cls[0].cpu().numpy())
    conf           = float(box.conf[0].cpu().numpy())

    # Draw rectangle
    cv2.rectangle(img, (x1,y1), (x2,y2), (0,255,0), 2)

    # Label text
    label = f"{model.names[cls_id]} {conf:.2f}"
    cv2.putText(
        img, label,
        (x1, max(y1-10,0)),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5,
        (0,255,0), 1, cv2.LINE_AA
    )

# 6. Show the annotated image
cv2.namedWindow("Detections", cv2.WINDOW_NORMAL)
cv2.imshow("Detections", img)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 384x640 8 player_lefts, 12 player_rights, 1 goalkeeper_left, 1 side_top, 1 side_left, 1 side_right, 1 rect_big_right_main, 1 rect_big_right_top, 1 rect_big_right_bottom, 1 rect_small_left_main, 19.3ms
Speed: 2.4ms preprocess, 19.3ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)


#### Reconstructed Lines and True Player Boxes

In [2]:
# Show Lines and boxes
from ultralytics import YOLO
import cv2
import numpy as np

# 1. Load trained offside model
model = YOLO('runs/train/offsidev32/weights/best.pt')

# 2. Load a sample image
img_path = '6.png'
img = cv2.imread(img_path)
h, w = img.shape[:2]
assert img is not None, f"Could not read {img_path}"

# 3. Inference
results = model(img, conf=0.25, imgsz=640)
res = results[0]

# 4. Separate boxes by class
# We'll keep only the highest‐conf box per player
raw = {}   # class_id -> (conf, (x1,y1,x2,y2))
lines    = []  # (class_id, (x1,y1,x2,y2))

plabels = {'player_left','player_right','goalkeeper_left','goalkeeper_right'}
raw = []
for box in res.boxes:
    x1,y1,x2,y2 = box.xyxy[0].cpu().numpy().astype(int)
    cls_id       = int(box.cls[0].cpu().numpy())
    name         = model.names[cls_id]
    conf         = float(box.conf[0].cpu().numpy())
    if name in plabels:
        raw.append((cls_id, name, conf, (x1,y1,x2,y2)))
    else:
        # line‐like classes
        lines.append((cls_id, (x1,y1,x2,y2), conf))

# 4a. Cluster overlapping boxes by simple greedy IoU grouping
def iou(a,b):
    x1,y1,x2,y2 = a
    X1,Y1,X2,Y2 = b
    xi1, yi1 = max(x1,X1), max(y1,Y1)
    xi2, yi2 = min(x2,X2), min(y2,Y2)
    if xi2<=xi1 or yi2<=yi1:
        return 0.0
    inter = (xi2-xi1)*(yi2-yi1)
    union = (x2-x1)*(y2-y1) + (X2-X1)*(Y2-Y1) - inter
    return inter/union

clusters = []
for cls_id,name,conf,box in sorted(raw, key=lambda x: -x[2]):
    placed=False
    for cl in clusters:
        # if it overlaps any in the cluster, add it there
        if any(iou(box, member[3])>0.5 for member in cl):
            cl.append((cls_id,name,conf,box))
            placed=True
            break
    if not placed:
        clusters.append([(cls_id,name,conf,box)])

# 4b. From each cluster pick the highest‐confidence detection and draw it
for cl in clusters:
    # they were sorted by conf already, so the first is best
    best_cls, best_name, best_conf, (x1,y1,x2,y2) = cl[0]
    color = (0,255,0) if best_name.startswith('player_') else (0,128,255)
    label = f"{best_name} {best_conf:.2f}"
    cv2.rectangle(img, (x1,y1), (x2,y2), color, 2)
    cv2.putText(img, label, (x1, y1-8),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

# 5. Draw lines
# define which classes are horizontal vs vertical
horizontal = {'side_top','side_bottom',
              'rect_big_left_top','rect_big_right_top',
              'rect_big_left_bottom','rect_big_right_bottom'}
vertical   = {'side_left','side_right',
              'rect_big_left_main','rect_big_right_main',
              'rect_small_left_main','rect_small_right_main'}
right_classes = ['side_right', 
                'rect_big_right_main',
                'rect_small_right_main']
left_classes = ['side_left', 
                'rect_big_left_main',
                'rect_small_left_main']
flag=False
#print(lines)
for cls_id, (x1,y1,x2,y2), conf in lines:
    name = model.names[cls_id]
    if name in right_classes:
        flag=True
        break
    elif name in left_classes:
        flag=False
        break
#print(flag)
for cls_id, (x1,y1,x2,y2), conf in lines:
    name = model.names[cls_id]
    color = (255,255,0)  # cyan for lines
    label = f"{name} {conf:.2f}"
    if flag:
        if name in horizontal:
            # bottom‐left → top‐right
            pt1 = (x1, y2)
            pt2 = (x2, y1)
        elif name in vertical:
            # top‐left → bottom‐right
            pt1 = (x1, y1)
            pt2 = (x2, y2)
        else:
            # fallback: draw its box
            pt1 = (x1, y1)
            pt2 = (x2, y2)
    else:
        if name in horizontal:
            # top‐left → bottom‐right
            pt1 = (x1, y1)
            pt2 = (x2, y2)
        elif name in vertical:
            # top‐left → bottom‐right
            pt1 = (x1, y2)
            pt2 = (x2, y1)
        else:
            # fallback: draw its box
            pt1 = (x1, y1)
            pt2 = (x2, y2)

    cv2.line(img, pt1, pt2, color, 3)
    cv2.putText(img, label, (pt1[0], pt1[1] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

# 6. Show
cv2.namedWindow("Offside Detections", cv2.WINDOW_NORMAL)
cv2.imshow("Offside Detections", img)
cv2.waitKey(0)
cv2.destroyAllWindows()



0: 384x640 8 player_lefts, 12 player_rights, 1 goalkeeper_left, 1 side_top, 1 side_left, 1 side_right, 1 rect_big_right_main, 1 rect_big_right_top, 1 rect_big_right_bottom, 1 rect_small_left_main, 10.5ms
Speed: 2.4ms preprocess, 10.5ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)


### Select Teams
##### First Select Player belonging to the left team then right
##### Press ESC after selecting the players

In [3]:
import cv2
import numpy as np
from ultralytics import YOLO

# 1. Run YOLO and collect all player boxes as before
model    = YOLO('runs/train/offsidev32/weights/best.pt')
img      = cv2.imread('6.png')
h, w     = img.shape[:2]
results  = model(img, conf=0.25, imgsz=640)[0]

# gather just the player boxes
player_boxes = []
for b in results.boxes:
    cls = model.names[int(b.cls[0].cpu())]
    if cls.startswith('player_'):
        x1,y1,x2,y2 = b.xyxy[0].cpu().numpy().astype(int)
        player_boxes.append((x1,y1,x2,y2))

# 2. Show these boxes and let the user click one "left‐team" and one "right‐team" example
selected = []  # will hold two entries: [(x1,y1,x2,y2), (x1,y1,x2,y2)]
clone = img.copy()

def on_click(event, x, y, flags, param):
    if event != cv2.EVENT_LBUTTONDOWN or len(selected) >= 2:
        return
    # find which box contains the click
    for box in player_boxes:
        x1,y1,x2,y2 = box
        if x1 < x < x2 and y1 < y < y2:
            selected.append(box)
            # draw a green or magenta rectangle around it as feedback
            color = (0,255,0) if len(selected)==1 else (255,0,255)
            cv2.rectangle(clone, (x1,y1),(x2,y2), color, 2)
            cv2.imshow("Select seeds", clone)
            break

cv2.namedWindow("Select seeds", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Select seeds",960,int(960*h/w))
cv2.imshow("Select seeds", clone)
cv2.setMouseCallback("Select seeds", on_click)

print("Please click once on a *player_left* box, then once on a *player_right* box.")
cv2.waitKey(0)
cv2.destroyWindow("Select seeds")

if len(selected) != 2:
    raise RuntimeError("You must click exactly two boxes!")

left_seed, right_seed = selected

# 3. Compute HSV histograms for each seed
def compute_hist(box):
    x1,y1,x2,y2 = box
    roi = img[y1:y2, x1:x2]
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    h_bins, s_bins = 30, 32
    hist = cv2.calcHist([hsv], [0,1], None, [h_bins, s_bins], [0,180, 0,256])
    cv2.normalize(hist, hist)
    return hist.flatten()

hist_left  = compute_hist(left_seed)
hist_right = compute_hist(right_seed)

# 4. Classify every detected player by histogram‐distance
def compare_hist(hist1, hist2):
    return cv2.compareHist(hist1.astype('float32'),
                           hist2.astype('float32'),
                           cv2.HISTCMP_BHATTACHARYYA)

final_boxes = []
for box in player_boxes:
    hist = compute_hist(box)
    d_left  = compare_hist(hist, hist_left)
    d_right = compare_hist(hist, hist_right)
    label   = 'player_left'  if d_left < d_right else 'player_right'
    final_boxes.append((box, label))

# 5. Draw them on the image
out = img.copy()

for (x1,y1,x2,y2), label in final_boxes:
    color = (0,255,0) if label=='player_left' else (255,0,255)
    cv2.rectangle(out, (x1,y1),(x2,y2), color, 2)
    cv2.putText(out, label, (x1, y1-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
    
scale = 960 / out.shape[1]
disp = cv2.resize(out, (960, int(out.shape[0]*scale)))

cv2.namedWindow("Team‐classified players", cv2.WINDOW_NORMAL)
cv2.imshow("Team‐classified players", disp)
cv2.waitKey(0)
cv2.destroyAllWindows()


0: 384x640 8 player_lefts, 12 player_rights, 1 goalkeeper_left, 1 side_top, 1 side_left, 1 side_right, 1 rect_big_right_main, 1 rect_big_right_top, 1 rect_big_right_bottom, 1 rect_small_left_main, 44.4ms
Speed: 5.0ms preprocess, 44.4ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)
Please click once on a *player_left* box, then once on a *player_right* box.


### Final Code (YOLO Detection + Team Classification + Offside Logic)

In [4]:
from ultralytics import YOLO
import cv2, numpy as np

# STEP 1: Team Selection
def select_teams_by_hist(img, player_boxes):
    # Let user pick two seeds in one window
    disp = img.copy()
    selected = []
    cv2.namedWindow("Pick LEFT, then RIGHT team seed", cv2.WINDOW_NORMAL)

    def on_click(evt, x, y, flags, param):
        nonlocal disp, selected
        if evt != cv2.EVENT_LBUTTONDOWN or len(selected) >= 2:
            return
        for box in player_boxes:
            x1,y1,x2,y2 = box
            if x1 < x < x2 and y1 < y < y2:
                selected.append(box)
                # draw a green or magenta rectangle around it as feedback
                clr = (0,255,0) if len(selected)==1 else (255,0,255)
                cv2.rectangle(disp,(x1,y1),(x2,y2), clr, 2)
                cv2.imshow("Pick LEFT, then RIGHT team seed", disp)
                break

    cv2.setMouseCallback("Pick LEFT, then RIGHT team seed", on_click)
    cv2.imshow("Pick LEFT, then RIGHT team seed", disp)
    while len(selected)<2:
        if cv2.waitKey(30)==27:
            cv2.destroyAllWindows()
            raise RuntimeError("Seed selection canceled")
    cv2.destroyAllWindows()

    left_seed, right_seed = selected

    # 2. Build histograms
    def make_hist(box):
        x1,y1,x2,y2 = box
        roi = img[y1:y2, x1:x2]
        hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
        hist = cv2.calcHist([hsv],[0,1],None,[30,32],[0,180,0,256])
        cv2.normalize(hist,hist)
        return hist.flatten()

    hist_L = make_hist(left_seed)
    hist_R = make_hist(right_seed)

    # 3. Classify every box by Bhattacharyya distance
    def bhatta(h1,h2):
        return cv2.compareHist(h1.astype('float32'), h2.astype('float32'),
                               cv2.HISTCMP_BHATTACHARYYA)

    players = []
    for box in player_boxes:
        hist = make_hist(box)
        dL = bhatta(hist, hist_L)
        dR = bhatta(hist, hist_R)
        label = 'player_left' if dL<dR else 'player_right'
        # confidence placeholder of 1.0
        players.append([(label, 1.0, box)])

    return players


# ——————————————————————————————————————————
# STEP 2: Offside Logic
def run_offside_on(frame, line_segs, players):
    h, w = frame.shape[:2]

    # 1. Pick touch_cls & horizontal_cls from line_segs.keys()
    line_classes = set(line_segs.keys())
    #print(line_classes)
    
    # priority: Side line → Big rect. → Small rect.
    if "side_right" in line_classes:
        touch_cls = "side_right"
    elif "side_left" in line_classes:
        touch_cls = "side_left"
    elif "rect_big_right_main" in line_classes:
        touch_cls = "rect_big_right_main"
    elif "rect_big_left_main" in line_classes:
        touch_cls = "rect_big_left_main"
    elif "rect_small_right_main" in line_classes:
        touch_cls = "rect_small_right_main"
    elif "rect_small_left_main" in line_classes:
        touch_cls = "rect_small_left_main"
    else:
        raise ValueError("No usable vertical field‐edge found in annotations")

    # always use Side line top as the horizontal reference
    if   "side_top"    in line_classes: horizontal_cls = "side_top"
    elif "side_bottom" in line_classes: horizontal_cls = "side_bottom"
    elif "rect_big_right_top" in line_classes: horizontal_cls = "rect_big_right_top"
    elif "rect_big_right_bottom" in line_classes: horizontal_cls = "rect_big_right_bottom"
    elif "rect_big_left_top" in line_classes: horizontal_cls = "rect_big_left_top"
    elif "rect_big_left_bottom" in line_classes: horizontal_cls = "rect_big_left_bottom"
    else:
        raise RuntimeError("No horizontal pitch edge found in annotations")
    
    #print(touch_cls)
    #print(horizontal_cls)

    # 2. Extract endpoints(A_top,B_top) & (A_side,B_side) from line_segs[touch_cls] etc.
    def endpoints_from_box(box, kind):
        (x1, y1), (x2, y2) = box
        if kind == 'horizontal':
            # bottom‐left → top‐right
            pt1 = np.array([x1, y2], dtype=float)
            pt2 = np.array([x2, y1], dtype=float)
        elif kind == 'vertical':
            # top‐left → bottom‐right
            pt1 = np.array([x1, y1], dtype=float)
            pt2 = np.array([x2, y2], dtype=float)
        else:
            raise ValueError("kind must be 'horizontal' or 'vertical'")
        return pt1, pt2
    #print(line_segs)
    h_box      = line_segs[horizontal_cls]
    #print(h_box)
    v_box      = line_segs[touch_cls]
    #print(v_box)

    A_top,  B_top  = endpoints_from_box(h_box, 'horizontal')
    #print(A_top, B_top)
    A_side, B_side = endpoints_from_box(v_box, 'vertical')
    #print(A_side, B_side)

    # draw references
    cv2.line(frame, tuple(A_top.astype(int)),  tuple(B_top.astype(int)),  (255,255,255),2)
    cv2.line(frame, tuple(A_side.astype(int)), tuple(B_side.astype(int)), (255,255,255),2)

    # 3. Extend touch-line to bottom or top so it spans full frame
    d_line = B_side - A_side
    if abs(d_line[1])>1e-6:
        # extend downward if right touch-line, upward if left
        target_y = h if touch_cls.endswith("right") else 0
        t = (target_y - A_side[1]) / d_line[1]
        B_ext = A_side + d_line * t
    else:
        B_ext = B_side.copy()
    cv2.line(frame, tuple(A_side.astype(int)), tuple(B_ext.astype(int)), (200,200,255),2)

    # 4. Pick defender & attacker by perpendicular distance to goal line or along a unit vector parallel to side line
    def pick_closest(team_cls, color):
        # Original side‐line vector & its length
        side_vec   = B_side - A_side
        side_len   = np.linalg.norm(side_vec)

        #print(side_len)
        #print(h)

        # Unit‐vector along top‐line for fallback
        u_top = (B_top - A_top)
        u_top = (u_top / np.linalg.norm(u_top)).astype(np.float32)

        # A threshold on the raw annotation length
        min_len = 0.5 * h   # e.g. 50% of frame height

        best = None
        for pl in players:
            name,_,(x1,y1,x2,y2) = pl[0]
            if name != team_cls:
                continue

            # choose correct foot‐x
            if touch_cls.endswith("right"):
                fx = x2
            else:
                fx = x1
            fy = y2
            P  = np.array([fx, fy], dtype=float)

            # If the annotated side‐line is long enough, use the true perp‐distance
            if side_len > min_len:
                dist = abs(np.cross(side_vec, P - A_side)) / side_len
            else:
                # fallback: project along the goal‐line direction
                dist = abs((P - A_side).dot(u_top))

            if best is None or dist < best[0]:
                best = (dist, P, (x1,y1,x2,y2))
        if best == None:
            raise RuntimeError("Players not detected for one or teams")
        # Draw the chosen box & foot
        d, foot, (x1,y1,x2,y2) = best
        cv2.rectangle(frame,(int(x1),int(y1)),(int(x2),int(y2)),color,2)
        cv2.circle(   frame,(int(foot[0]),int(foot[1])),6,color,-1)

        return best

    # defender = same-side team as touch-line (if right touch-line, defender team right)
    def_team = "player_right" if touch_cls=="side_right" else "player_left"
    att_team = "player_left"  if touch_cls=="side_right" else "player_right"
    dist_def, def_foot, _ = pick_closest(def_team,(0,255,0))
    dist_att, att_foot, _ = pick_closest(att_team,(0,255,255))

    # 5. Compute vanishing point V for “vertical” references
    vert_classes = {touch_cls, 
                    "rect_big_right_main" if touch_cls.endswith("right") else "rect_big_left_main",
                    "rect_small_right_main" if touch_cls.endswith("right") else "rect_small_left_main"}
    lines = []
    #print(line_segs.items())
    for cl, ln in line_segs.items():
        if cl in vert_classes:
            pts = np.array(ln,float).reshape(-1,2)
            A,B = pts[pts[:,1].argmin()], pts[pts[:,1].argmax()]
            lines.append(np.cross(np.append(A,1), np.append(B,1)))
    L = np.stack(lines,axis=0)
    _,_,vh = np.linalg.svd(L)
    V = vh[-1]; V/=V[2]
    vx, vy = V[0], V[1]

    # 6. Draw offside lines (parallel to touch-line) through each foot ---
    Lspan = max(h,w)*1.5
    for foot, col in [(def_foot,(0,0,255)), (att_foot,(255,0,255))]:
        dir_img = np.array([vx,vy]) - foot
        dir_img /= np.linalg.norm(dir_img)
        P1 = foot - dir_img*Lspan
        P2 = foot + dir_img*Lspan
        cv2.line(frame, tuple(P1.astype(int)), tuple(P2.astype(int)), col,3)

    # 7. OFFSIDE decision
    if dist_att < dist_def:
        cv2.putText(frame,"OFFSIDE",(50,80),
                    cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),4)

    return frame

# ——————————————————————————————————————————
# STEP 3: Wrapper that runs YOLO and builds those dicts

def detect_and_offside(img_path, yolo_weights):
    # 1. Read
    frame = cv2.imread(img_path)
    h, w = frame.shape[:2]

    # 2. Run YOLO
    model   = YOLO(yolo_weights)
    results = model(frame, conf=0.25, imgsz=640)[0]

    # 3. Extract line segments & players
    # line-like classes map
    horizontal = {
        'side_top','side_bottom',
        'rect_big_left_top','rect_big_right_top',
        'rect_big_left_bottom','rect_big_right_bottom'
    }
    vertical   = {
        'side_left','side_right',
        'rect_big_left_main','rect_big_right_main',
        'rect_small_left_main','rect_small_right_main'
    }

    right_classes = {
        'side_right', 
        'rect_big_right_main',
        'rect_small_right_main'
        }
    left_classes = {
        'side_left', 
        'rect_big_left_main',
        'rect_small_left_main'
        }

    line_boxes = {}
    raw_players = []
    for box in results.boxes:
        x1,y1,x2,y2 = box.xyxy[0].cpu().numpy().astype(int)
        cls_id      = int(box.cls[0].cpu().numpy())
        name        = model.names[cls_id]
        conf        = float(box.conf[0].cpu().numpy())

        if name in horizontal or name in vertical:
            # keep only the most confident segment per class
            prev = line_boxes.get(name)
            if prev is None or conf>prev[1]:
                line_boxes[name] = ((x1,y1,x2,y2), conf)
        else:
            # gather all player/keeper boxes for NMS‐across‐classes
            raw_players.append((name, conf, (x1,y1,x2,y2)))
    #print(raw_players)
    #print(line_boxes)

    flag=False
    #print(lines)
    for name, (box, conf) in line_boxes.items():
        if name in right_classes:
            flag=True
            break
        elif name in left_classes:
            flag=False
            break
    #print(flag)    

    line_segs = {}
    #print(line_boxes.items())
    for name, (box, conf) in line_boxes.items():
        #print(box)
        (x1, y1, x2, y2) = box
        if flag:
            if name in horizontal:
                # bottom‐left → top‐right
                pt1 = (x1, y1)
                pt2 = (x2, y2)
            elif name in vertical:
                # top‐left → bottom‐right
                pt1 = (x1, y1)
                pt2 = (x2, y2)
            else:
                # fallback: draw its box
                pt1 = (x1, y1)
                pt2 = (x2, y2)
        else:
            if name in horizontal:
                # top‐left → bottom‐right
                pt1 = (x1, y2)
                pt2 = (x2, y1)
            elif name in vertical:
                # bottom‐left → top‐right
                pt1 = (x1, y2)
                pt2 = (x2, y1)
            else:
                # fallback: draw its box
                pt1 = (x1, y1)
                pt2 = (x2, y2)

        # store the one true segment for this class
        line_segs[name] = (pt1, pt2)

    # 4. Do simple across‐class NMS for players
    #    to get final list of (name, (fx,fy)) foot points:
    clusters = []
    def iou(a,b):
        x1,y1,x2,y2 = a; X1,Y1,X2,Y2=b
        xi1, yi1 = max(x1,X1), max(y1,Y1)
        xi2, yi2 = min(x2,X2), min(y2,Y2)
        if xi2<=xi1 or yi2<=yi1: return 0.0
        inter = (xi2-xi1)*(yi2-yi1)
        union = (x2-x1)*(y2-y1)+(X2-X1)*(Y2-Y1)-inter
        return inter/union

    for name,conf,box in sorted(raw_players, key=lambda x:-x[1]):
        placed=False
        for cl in clusters:
            if any(iou(box,m[2])>0.5 for m in cl):
                cl.append((name,conf,box))
                #cl.append(box)
                placed=True
                break
        if not placed:
            clusters.append([(name,conf,box)])
            #clusters.append(box)
    #print(clusters)
    # Remove any cluster whose best label starts with "goalkeeper"
    clusters = [
        cl
        for cl in clusters
        if not cl[0][0].startswith("goalkeeper")
    ]
    player_boxes = [ cluster[0][2] for cluster in clusters ]
    players = select_teams_by_hist(frame, player_boxes)

    # 5. call your offside routine
    out = run_offside_on(frame, line_segs, players)

    # 6. Show
    cv2.namedWindow("Offside", cv2.WINDOW_NORMAL)
    cv2.imshow("Offside", out)
    cv2.resizeWindow("Offside",960,int(960*h/w))
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# ——————————————————————————————————————————
# MAIN
if __name__ == "__main__":
    detect_and_offside("1.png", "runs/train/offsidev32/weights/best.pt")


0: 384x640 9 player_lefts, 3 player_rights, 1 goalkeeper_right, 1 side_top, 1 side_left, 1 rect_big_left_main, 1 rect_big_left_top, 1 rect_big_right_main, 1 rect_big_right_top, 1 rect_small_left_main, 1 rect_small_right_main, 34.7ms
Speed: 5.6ms preprocess, 34.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)
